# 0. 분석 방향
* 예측은 **유저별** 생존 기간, 평균 결제 금액이기 때문에 유저별로 groupby를 어떻게 진행할지에 관점을 두고 볼 것이다.
* train, test1, test2의 데이터 분포와 통계치를 확인할 것이다.<BR>(train을 학습하여 test1과 test2를 예측하는데 데이터 간의 분포와 통계치가 많은 차이가 있을 경우 예측에 도움이 안될 것이다.)

# 1. 도입(라이브러리, 데이터 로딩)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy as sp

import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
     
import os
%matplotlib inline

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
os.chdir('..') #상위디렉터리로 이동
path_dir = os.getcwd()+"/champions/"
file_list = sorted(os.listdir(path_dir))

data_dict = OrderedDict()
for file in file_list:
    if file.find('csv') is not -1:
        if file.split('_')[1] == 'trade.csv':
            data_dict[file.split('.')[0]] = pd.read_csv(path_dir+file)

## 1.1 자료 구분값 추가(load 파일 구분)

In [3]:
data_dict['train_trade']['kind'] = 'train'
data_dict['test1_trade']['kind'] = 'test1'
data_dict['test2_trade']['kind'] = 'test2'
data_dict['train_trade']['kind_int'] = 1
data_dict['test1_trade']['kind_int'] = 2
data_dict['test2_trade']['kind_int'] = 3

trade_all = data_dict['train_trade'].append(data_dict['test1_trade'])
del(data_dict['train_trade'])
del(data_dict['test1_trade'])
trade_all = trade_all.append(data_dict['test2_trade'])
del(data_dict['test2_trade'])

In [4]:
trade_all.shape

(3285150, 13)

In [5]:
trade_all.groupby('kind')['day'].count()

kind
test1     735652
test2     700173
train    1849325
Name: day, dtype: int64

In [6]:
trade_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3285150 entries, 0 to 700172
Data columns (total 13 columns):
day               int64
time              object
type              int64
server            object
source_acc_id     int64
source_char_id    int64
target_acc_id     int64
target_char_id    int64
item_type         object
item_amount       float64
item_price        float64
kind              object
type_int          int64
dtypes: float64(2), int64(7), object(4)
memory usage: 350.9+ MB


In [40]:
trade_all.isna().sum()

day                     0
time                    0
type                    0
server                  0
source_acc_id           0
source_char_id          0
target_acc_id           0
target_char_id          0
item_type               0
item_amount             0
item_price        1968180
kind                    0
type_int                0
dtype: int64

In [7]:
trade_all.head(10)

,day,time,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price,kind,type_int
0,7,21:13:05,1,ag,11439,385109,48152,34247,enchant_scroll,4.793968e-08,NaN,train,1
1,2,15:33:00,1,al,22220,259705,43228,379826,enchant_scroll,4.314571e-06,NaN,train,1
2,1,14:24:53,1,al,18212,164230,38145,34209,accessory,7.190952e-08,NaN,train,1
3,7,23:13:43,1,au,121622,49839,71226,281334,adena,9.587936e-03,NaN,train,1
4,2,12:43:33,0,bd,14421,344503,65220,210558,etc,1.198492e-07,0.000639,train,1
5,2,16:56:12,0,aj,90975,346488,22407,110658,enchant_scroll,7.190952e-08,0.005693,train,1
6,2,21:54:22,1,av,10578,286509,52677,365997,etc,4.793968e-05,NaN,train,1
7,7,14:14:46,0,bc,96563,209660,82644,54832,etc,1.198492e-07,0.067717,train,1
8,6,02:29:14,1,av,107975,5446,13885,181430,etc,7.670349e-06,NaN,train,1
9,6,02:29:02,0,as,91572,154600,41504,4697,etc,4.793968e-08,0.183391,train,1


# 2. 탐색적 자료 분석

In [8]:
print("source_acc_id 총 개수 : {}".format(trade_all['source_acc_id'].size))
print("target_char_id 총 개수 : {}".format(trade_all['target_char_id'].size))
print("source_acc_id의 유니크 개수 : {}".format(trade_all['source_acc_id'].unique().size))
print("target_char_id의 유니크 개수 : {}".format(trade_all['target_char_id'].unique().size))

source_acc_id 총 개수 : 3285150
target_char_id 총 개수 : 3285150
source_acc_id의 유니크 개수 : 86158
target_char_id의 유니크 개수 : 173263


In [39]:
sum(trade_all['source_acc_id'] == trade_all['target_acc_id'])

0

* source_acc_id != target_acc_id 을 알 수 있다.
* 상식대로(?) 내가 나와 거래는 할 수 없다.

# 3. Feature Engineering

**FE를 통해 추가한 변수 리스트**

* 기록된 day 변수의 개수와 리스트
    * **day_sum_count_trade** : 개수
    * **day_sum_detail_trade** : 리스트
* 활동중인 서버, 서버리스트
    * **server_sum_count_trade** : 개수
    * **server_sum_detail_trade** : 리스트
* 거래한 케릭터 개수와 리스트
    * **char_sum_count_trade** : 개수
    * **char_sum_detail_trade** : 리스트
* (교환+개인상점)을 했던 유저 수와 리스트
    * **trade_accid_account** : 개수
    * **trade_accid_detail** : 리스트
* **type_1_accid_account** : 거래한 유저의 수
* **trade_source_count** : 교환창으로 주거나/판매한 횟수
* **trade_target_count** : 교환창으로 받거나/구매한 횟수
* **type_1** : 교환창을 통해 주거나/판매한 횟수 + 받거나/구매한 횟수
* **type_0_source_count** : 상점 거래로 판매한 횟수
* **type_0_target_count** : 상점 거래로 구매한 횟수
* **type_0** : 상점을 통해 판매한 횟수 + 구매한 횟수
* **type_total_sum** : 교환+상점을 통해 거래한 횟수
* **type_0_is_open** : 상점 오픈 유무
* 아이템을 받거나 구매한 양
    * **item_accessory_count** : 액세서리 개수
    * **item_adena_count** : 아데나 개수
    * **item_armor_count** : 방어구 개수
    * **item_enchant_scroll_count** : 주문서 개수
    * **item_etc_count** : 기타 개수
    * **item_spell_count** : 스킬북 개수
    * **item_weapon_count** : 무기 개수
* 아이템을 팔거나 준것
    * **item_accessory_out_count** : 액세서리 개수
    * **item_adena_out_count** : 아데나 개수
    * **item_armor_out_count** : 방어구 개수
    * **item_enchant_scroll_out_count** : 주문서 개수
    * **item_etc_out_count** : 기타 개수
    * **item_spell_out_count** : 스킬북 개수
    * **item_weapon_out_count** : 무기 개수
* **type_0_avenue** : 상점 수입
* **not_any_trade** : 거래 유무